We're going to try three scenarios. Fitting to:

1. a full cone
2. a partial cone cut, from the outside of a plane parallel to the cone axis
3. another partial cone cut, but now an ellipsoidal one, or rather a sort of "thick" ellipse, with thickness along the cone directrices

In [ ]:
import numpy as np
import ipyvolume as ipv
import ectopylasm as ep

import symfit as sf

In [ ]:
cone = ep.geometry.Cone(0.5, 0.5)

# 1. Full cone fit

In [ ]:
n_steps = 20
xyz = np.array(ep.geometry.cone_surface(cone, n_steps=n_steps))
xyz = xyz.reshape(3, n_steps*n_steps)
xyz += np.random.normal(0, 0.03, xyz.shape)

Implicit model for the cone from [Wolfram Mathworld](http://mathworld.wolfram.com/Cone.html):

$$
\frac{x^2 + y^2}{c^2} = (z - z_0)^2 \,,
$$

where $c=r/h$ and $z_0=h$. This is for a cone along the z-axis, with the base (center of the circle) at the origin.

To fit to a cone with generic rotation and location, we need to translate from the actual space coordinates of points x, y, z back to this nice space where the implicit cone equation holds. Let's call the latter space u, v, w, so that now the cone equation is

$$
h^2 \frac{u^2 + v^2}{r^2} = (w - h)^2 \,.
$$

We can then write the transformation between $\vec{x} = (x, y, z)$ and $\vec{u} = (u, v, w)$ as

$$
\vec{u} = M_x(-\theta_x) M_y(-\theta_y) (\vec{x} - \vec{b}) \,,
$$

where $\vec{b}$ is the base position, the $\theta$s are the rotation angles of the cone around the two axes and the $M$s are the rotation matrices around those axes.

In [ ]:
h, radius, rot_x, rot_y, bx, by, bz = sf.parameters('h, radius, rot_x, rot_y, bx, by, bz')
x, y, z, f = sf.variables('x, y, z, f')

x_min_b = sf.Matrix([x - bx, y - by, z - bz])  # column matrix

M_x = sf.Matrix([[1, 0, 0],
                 [0, sf.cos(-rot_x), sf.sin(-rot_x)],
                 [0, -sf.sin(-rot_x), sf.cos(-rot_x)]])

M_y = sf.Matrix([[sf.cos(-rot_y), 0, sf.sin(-rot_y)],
                 [0, 1, 0],
                 [-sf.sin(-rot_y), 0, sf.cos(-rot_y)]])

u = M_x @ (M_y @ x_min_b)

cone_model = {
    f: h**2 * (u[0]**2 + u[1]**2) / radius**2 - (u[2] - h)**2
}

cone_fit = sf.Fit(cone_model, x=xyz[0], y=xyz[1], z=xyz[2], f=np.zeros_like(xyz[0]))

cone_fit_result = cone_fit.execute()

print(cone_fit_result)

In [ ]:
cone_fit_result.params['radius']

In [ ]:
def plot_cone_fit(result):
    cone = ep.geometry.Cone(result.params['h'], result.params['radius'],
                            rot_x=result.params['rot_x'], rot_y=result.params['rot_y'],
                            base_pos=ep.geometry.Point(
                                result.params['bx'],
                                result.params['by'],
                                result.params['bz']
                            ))
    fig = ep.visualize.plot_cone(cone)
    return fig

In [ ]:
ipv.clear()
ipv.scatter(*xyz)
fit = cone_fit_result
plot_cone_fit(fit)
ipv.show()

That's not too great.

Maybe some initial guesses would help. For instance, radius and height can be guessed using the maximum distance... but that itself is not easy to compute automatically (convex hull or something like that). Easier way: just compute the min and max in all directions and use that box's sizes as estimates.

In [ ]:
xyz_bounding_size = sorted(xyz.ptp(axis=1))

h.value = xyz_bounding_size[-1]
radius.value = xyz_bounding_size[-2]/2

cone_fit = sf.Fit(cone_model, x=xyz[0], y=xyz[1], z=xyz[2], f=np.zeros_like(xyz[0]))

# reset values for later attempts
h.value, radius.value = 1, 1

cone_fit_result = cone_fit.execute()

print(cone_fit_result)

ipv.clear()
ipv.scatter(*xyz)
plot_cone_fit(cone_fit_result)
ipv.show()

Still doesn't work, basically same result. Maybe we should also sensibly initialize the other parameters.

But let's first try with a less noisy dataset.

In [ ]:
n_steps = 30
xyz = np.array(ep.geometry.cone_surface(cone, n_steps=n_steps))
xyz = xyz.reshape(3, n_steps*n_steps)
# xyz += np.random.normal(0, 0.01, xyz.shape)

In [ ]:
cone_fit = sf.Fit(cone_model, x=xyz[0], y=xyz[1], z=xyz[2], f=np.zeros_like(xyz[0]))

cone_fit_result = cone_fit.execute()

print(cone_fit_result)

ipv.clear()
ipv.scatter(*xyz)
plot_cone_fit(cone_fit_result)
ipv.show()

It's kinda suspicious that rot_x is almost exactly $\pi$ all the time...

What if we set the initial guesses to the ideal values?

In [ ]:
h.value = 0.5
radius.value = 0.5
rot_x.value = 0
rot_y.value = 0
bx.value = 0
by.value = 0
bz.value = 0

cone_fit = sf.Fit(cone_model, x=xyz[0], y=xyz[1], z=xyz[2], f=np.zeros_like(xyz[0]))

cone_fit_result = cone_fit.execute()

# reset values for later attempts
for ding in (h, radius, rot_x, rot_y, bx, by, bz):
    ding.value = 1

print(cone_fit_result)

ipv.clear()
ipv.scatter(*xyz)
plot_cone_fit(cone_fit_result)
ipv.show()

Ok, at least it agrees that this is a good fit :)

Then, let's add some noise again.

In [ ]:
n_steps = 20
xyz = np.array(ep.geometry.cone_surface(cone, n_steps=n_steps))
xyz = xyz.reshape(3, n_steps*n_steps)
xyz += np.random.normal(0, 0.03, xyz.shape)

In [ ]:
h.value = 0.5
radius.value = 0.5
rot_x.value = 0
rot_y.value = 0
bx.value = 0
by.value = 0
bz.value = 0

cone_fit = sf.Fit(cone_model, x=xyz[0], y=xyz[1], z=xyz[2], f=np.zeros_like(xyz[0]))

cone_fit_result = cone_fit.execute()

# reset values for later attempts
for ding in (h, radius, rot_x, rot_y, bx, by, bz):
    ding.value = 1

print(cone_fit_result)

ipv.clear()
ipv.scatter(*xyz)
plot_cone_fit(cone_fit_result)
ipv.show()

Ok, this is visually quite crappy. It seems to be quite hard to fit points to a cone then...